In [56]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

#create database
kill_match_db = create_engine('sqlite:///kill_match.db')
df1 = pd.read_sql_query('select killed_by, killer_position_x, killer_position_y, victim_position_x, victim_position_y from kmatch where killed_by not like "%death.%" and killed_by not like "%Down%" and killed_by not like "%Blue%" and killed_by not like "%Fall%" and killed_by like "S12K" limit 100000', kill_match_db)




In [57]:
df1['kill_dist'] = ((df1['killer_position_x'] - df1['victim_position_x'])**2 + 
                    (df1['killer_position_y'] - df1['victim_position_y'])**2)**(1/2)
df1

,killed_by,killer_position_x,killer_position_y,victim_position_x,victim_position_y,kill_dist
0,S12K,371186.5,423791.3,371092.2,423650.7,169.295156
1,S12K,438757.9,280729.9,439254.2,280726.3,496.313056
2,S12K,428125.0,528350.6,428281.9,528305.8,163.170616
3,S12K,432168.4,636683.8,432251.5,636724.3,92.443821
4,S12K,342712.8,173653.6,342536.6,173721.0,188.651001
5,S12K,364785.9,417538.5,365002.5,417429.4,242.524988
6,S12K,418760.3,327511.5,417498.9,326726.4,1485.769824
7,S12K,419011.3,328593.6,418930.0,328264.4,339.090445
8,S12K,239503.8,524631.9,239917.9,523771.3,955.045114
9,S12K,627460.9,383382.3,627531.8,383342.1,81.503681


In [58]:
def iterate(k):
    cluster_centers = []
    convergence = 0
    
    #Random choose k centers
    for i in range (k):
        index = np.random.choice(len(df1['kill_dist']))
        cluster_centers.append(round(df1.loc[index,'kill_dist']))
        
    while convergence !=k:
        convergence = 0
        groups = []
        
        #find the closest center for each point
        for i in range(len(df1['kill_dist'])):
            min_dist = abs(df1.loc[i,'kill_dist'] - cluster_centers[0])
            min_ind = 0
            for j in range(1,k):
                temp_dist = abs(df1.loc[i,'kill_dist'] - cluster_centers[j])
                if temp_dist < min_dist:
                    min_dist = temp_dist
                    min_ind = j
            groups.append(min_ind)
        
        #calculate the new center
        for i in range (k):
            dsum = 0
            count = 0
            for j in range(len(df1['kill_dist'])):
                if groups[j] == i:
                    dsum += df1.loc[j,'kill_dist']
                    count += 1
            new_center = dsum/count
            if (new_center == cluster_centers[i]):
                convergence += 1
            else:
                cluster_centers[i] = new_center
        #print ("Score:",convergence)
        #print ("Centroids:",cluster_centers)
    #print ("Group:", groups)
    print ("Final cluster centers:",cluster_centers)
    count0 = 0
    count1 = 0
    count2 = 0
    for i in range(len(df1['kill_dist'])):
        if groups[i] == 0:
            count0 += 1
        elif groups[i] == 1:
            count1 += 1
        else:
            count2 += 1
    print ("Number of datapoint in cluster 0 is", count0)
    print ("Number of datapoint in cluster 1 is", count1)
    print ("Number of datapoint in cluster 2 is", count2)
            
                


In [59]:
iterate(3)

Final cluster centers: [50119.276720932845, 598.8230342361303, 5228.1326309995038]
Number of datapoint in cluster 0 is 39
Number of datapoint in cluster 1 is 97691
Number of datapoint in cluster 2 is 2270
